<a href="https://www.kaggle.com/code/yunusemreturkoglu/forecasting-energy-consumption-gdz?scriptVersionId=144893083" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, cross_validate
from sklearn import model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from warnings import filterwarnings
filterwarnings('ignore')
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
import matplotlib.pylab as plt
import matplotlib.dates as mdates

In [ ]:
df_train = pd.read_csv("/kaggle/input/train-gdz/train.csv")
df_train["Tarih"] = pd.to_datetime(df_train["Tarih"])
df_med = pd.read_csv("/kaggle/input/med-gdz/med.csv")
df_med['Tarih'] = pd.to_datetime(df_med['Tarih'])
df_sample = pd.read_csv('/kaggle/input/sample-gdz/sample_submission.csv')
df_sample["Tarih"] = pd.to_datetime(df_sample['Tarih'])
df_weather = pd.read_csv('/kaggle/input/hava-sicakligi/Izmir1.csv')
df_calender = pd.read_csv('/kaggle/input/calender-tr/Calendar.csv')


In [ ]:
color_pal = sns.color_palette()
df_train.plot(style='.', x='Tarih', y='Dağıtılan Enerji (MWh)', figsize=(10,5), ms=2, color=color_pal[0], title='consumption')
plt.show()

In [ ]:
df_train.columns = [col.lower() for col in df_train.columns]
df_med.columns = [col.lower() for col in df_med.columns]
df_sample.columns = [col.lower() for col in df_sample.columns]

In [ ]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
                            ordered=True)

In [ ]:
def mevsimler(ay):
    if ay in [12,1,2]:
        return "Kis"
    elif ay in[3,4,5]:
        return "Ilkbahar"
    elif ay in [6,7,8]:
        return "Yaz"
    else:
        return"Sonbahar"

In [ ]:
def create_features(df):
    df["saat"] = df["tarih"].dt.hour
    df["haftanin_gunu"] = df["tarih"].dt.dayofweek
    df["gunler"] = df["tarih"].dt.day_name()
    df["gunler"] = df["gunler"].astype(cat_type)
    df["ceyreklik"] = df['tarih'].dt.quarter
    df["ay"] = df['tarih'].dt.month
    df['yil'] = df['tarih'].dt.year
    df['yilin_gunleri'] = df['tarih'].dt.dayofyear
    df['ayin_gunleri'] = df['tarih'].dt.day
    df['yilin_haftalari'] = df['tarih'].dt.weekofyear
    df["mevsim"] = df["ay"].apply(mevsimler)

In [ ]:
create_features(df_train)
create_features(df_med)
create_features(df_sample)

In [ ]:
df_train["kesinti_gunleri"] = df_train.apply(lambda x: int((x[["yil", "ay", "ayin_gunleri"]] == df_med[["yil", "ay", "ayin_gunleri"]]).all(axis=1).any()), axis=1)

In [ ]:
df_train.head()

In [ ]:
df_med.head()

In [ ]:
df_sample.head()

In [ ]:
df_train["kesinti_gunleri"].value_counts()

In [ ]:
df_calender.head()

In [ ]:
df_calender["PUBLIC_HOLIDAY_FLAG"] = df_calender["PUBLIC_HOLIDAY_FLAG"].replace(["N"], 0)
df_calender["PUBLIC_HOLIDAY_FLAG"] = df_calender["PUBLIC_HOLIDAY_FLAG"].replace(["Y"], 1)
df_calender["RAMADAN_FLAG"] = df_calender["RAMADAN_FLAG"].replace(["N"], 0)
df_calender["RAMADAN_FLAG"] = df_calender["RAMADAN_FLAG"].replace(["Y"], 1)
df_calender["yil"] = pd.to_datetime(df_calender["CALENDAR_DATE"]).dt.year
df_calender["ay"] = pd.to_datetime(df_calender['CALENDAR_DATE']).dt.month
df_calender["ayin_gunleri"] = pd.to_datetime(df_calender['CALENDAR_DATE']).dt.day
df_calender = df_calender.drop(["CALENDAR_DATE", "CALENDAR_SK", "DAY_OF_MONTH", "DAY_OF_WEEK_SK", "SEASON_SK", "QUARTER_OF_YEAR",
                                "MONTH_SK", "WEEKEND_FLAG", "WEEK_OF_YEAR", "RELIGIOUS_DAY_FLAG_SK", "NATIONAL_DAY_FLAG_SK",
                                "SPECIAL_DAY_SK", "SPECIAL_DAY_SK2"], axis=1)

In [ ]:
df_calender = df_calender.iloc[853:2557]

In [ ]:
df_calendar_sample = df_calender[0:31]
df_calendar_train = df_calender[31:1704]

In [ ]:
mergedf_train = pd.merge(df_train, df_calendar_train, on=["yil", "ay", "ayin_gunleri"])
mergedf_sample = pd.merge(df_sample, df_calendar_sample, on=["yil", "ay", "ayin_gunleri"])
mergedf_train.rename(columns={"RAMADAN_FLAG": "ramazan"}, inplace=True)
mergedf_train.rename(columns={"PUBLIC_HOLIDAY_FLAG": "tatil"}, inplace=True)
mergedf_sample.rename(columns={"PUBLIC_HOLIDAY_FLAG": "tatil"}, inplace=True)
mergedf_sample.rename(columns={"RAMADAN_FLAG": "ramazan"}, inplace=True)

In [ ]:
# exporting train and sample 
mergedf_train.to_csv('TrainMe.csv')
mergedf_sample.to_csv('SampleMe.csv')

In [ ]:
# importing new datasets

df_sample = pd.read_csv("/kaggle/input/sample-gedizme/SampleMe.csv", index_col=0)
df_train = pd.read_csv("/kaggle/input/train-gedizme/TrainMe.csv", index_col=0)
df_train.rename(columns={"dağıtılan enerji (mwh)": "enerji"}, inplace=True)
df_sample.rename(columns={"dağıtılan enerji (mwh)": "enerji"}, inplace=True)
df_train["isFuture"] = False
df_sample["isFuture"] = True
df_train['tarih'] = pd.to_datetime(df_train['tarih'])
df_sample["tarih"] = pd.to_datetime(df_sample["tarih"])
df_sample['kesinti_gunleri'] = 0

In [ ]:
sns.boxplot(y=df_train["enerji"])

df_train.plot(style='.',x='yil', y="enerji", figsize=(15,5), color=color_pal[0], title="electric consumption")
plt.show()

In [ ]:
df_train["enerji"].plot(kind="hist", bins=500)

In [ ]:
# Creating "vakitler" variable

df_train.loc[(df_train["saat"]>=0) & (df_train['saat']<6), "vakitler"] = "gece"
df_train.loc[(df_train["saat"]>=6) & (df_train['saat']<=12), "vakitler"] = "sabah"
df_train.loc[(df_train["saat"]>12) & (df_train['saat']<16), "vakitler"] = "ogle"
df_train.loc[(df_train["saat"]>=16) & (df_train['saat']<19), "vakitler"] = "ikindi"
df_train.loc[(df_train["saat"]>=19) & (df_train['saat']<=23), "vakitler"] = "aksam"

In [ ]:
df_sample.loc[(df_sample["saat"]>=0) & (df_sample['saat']<6), "vakitler"] = "gece"
df_sample.loc[(df_sample["saat"]>=6) & (df_sample['saat']<=12), "vakitler"] = "sabah"
df_sample.loc[(df_sample["saat"]>12) & (df_sample['saat']<16), "vakitler"] = "ogle"
df_sample.loc[(df_sample["saat"]>=16) & (df_sample['saat']<19), "vakitler"] = "ikindi"
df_sample.loc[(df_sample["saat"]>=19) & (df_sample['saat']<=23), "vakitler"] = "aksam"

In [ ]:
# Merge train and sample datasets
df_train_sample = pd.concat([df_train, df_sample], ignore_index=True)
df_train_sample["tarih"] = pd.to_datetime(df_train_sample["tarih"])

In [ ]:
df_train_sample.info()

In [ ]:
# Adding weather features
df_train_sample["sicaklik"] = df_weather["temperature_2m (°C)"]
df_train_sample["nem"] = df_weather["relativehumidity_2m (%)"]
df_train_sample["ruzgar_hizi"] = df_weather["windspeed_100m (km/h)"]
df_train_sample["toprak_sicakligi"] = df_weather["soil_temperature_0_to_7cm (°C)"]
df_train_sample["toprak_nemi"] = df_weather["soil_moisture_0_to_7cm (m³/m³)"]


In [ ]:
# Creating new features related to weather

df_train_sample["sicaklik_nem"] = df_train_sample["sicaklik"]*df_train_sample["nem"]
df_train_sample["toprak_sicaklik_nem"] = df_train_sample['toprak_sicakligi']*df_train_sample['toprak_nemi']

In [ ]:
# Sin/Cos function for hour(extract useful information from cyclical data)

df_train_sample["saat_sin"] = np.sin(df_train_sample["saat"])
df_train_sample["saat_cos"] = np.cos(df_train_sample["saat"])

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
sp = ax.scatter(df_train_sample["saat_sin"], df_train_sample["saat_cos"], c=df_train_sample["saat"])
ax.set(xlabel = "sin(hour)", ylabel = "cos(hour)")

In [ ]:
# Adding GDP Feature

df_tuik = pd.read_csv("/kaggle/input/gdp-turkeyizmir/GDPA1.csv")
df_tuik.head()
df_tuik = df_tuik.set_index("DATE")
df_tuik.index = pd.to_datetime(df_tuik.index)
df_hourly = df_tuik.resample("H").interpolate(method="linear")

In [ ]:
df_hourly = df_hourly.iloc[:40896]
df_hourly = df_hourly.reset_index()
df_train_sample["GDP"] = df_hourly["NGDPRSAXDCTRQ"]

In [ ]:
# Creating Lag futures
df_train_sample["lag1"] = df_train_sample["enerji"].shift(728)
df_train_sample["lag2"] = df_train_sample["enerji"].shift(1092)
df_train_sample["lag3"] = df_train_sample["enerji"].shift(1456)
df_train_sample = df_train_sample.dropna()

In [ ]:
df_train_sample.head()

In [ ]:
# One-hot encoder for categorical variables

cat_cols = [col for col in df_train_sample.columns if df_train_sample[col].dtypes == "O"]
df_train_sample = pd.get_dummies(df_train_sample, columns=cat_cols, drop_first=True)

In [ ]:
# Robust scaler for numerical variables

num_cols = [col for col in df_train_sample.columns if df_train_sample[col].dtypes != "0" ]
num_cols.remove("tarih")
num_cols.remove("enerji")
num_cols.remove("isFuture")
num_cols = [col for col in num_cols if col not in cat_cols]

rs = RobustScaler()
df_train_sample[num_cols] = rs.fit_transform(df_train_sample[num_cols])

In [ ]:
df_train_sample.head()

In [ ]:
# Analysing correlation between the variables

fig, ax = plt.subplots(figsize=(10,10))
corr = df_train_sample.corr()
sns.heatmap(corr, xticklabels=corr.columns,
                    yticklabels=corr.columns, ax=ax)

In [ ]:
# Seperation into train and predict

df_train1 = df_train_sample.query("isFuture == False").copy()
df_train1.drop("isFuture", inplace=True, axis=1)

In [ ]:
df_predict = df_train_sample.query("isFuture").copy()
df_predict.reset_index(drop=True, inplace=True)
df_predict.drop("isFuture", inplace=True, axis=1)

## Looking Some Graphs

In [ ]:
# Analysing to outliers graph monthly

fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(data=df_train1, x="haftanin_gunu", y="enerji", hue="ay", ax=ax, linewidth=1)
ax.set_title("Power use energy day of week")
ax.set_xlabel("day of week")
ax.set_ylabel("energy")
ax.legend(bbox_to_anchor=(1,1))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df_train1, x='saat', y='enerji')
ax.set_title('enerji saatlik')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df_train1, x='ay', y='enerji')
ax.set_title('enerji aylik')
plt.show()

In [ ]:
color_pal = sns.color_palette()
df_train1.plot(style='.', x='tarih', y='enerji', figsize=(10,5), ms=2, color=color_pal[0], title='consumption')
plt.show()

In [ ]:
df_train1["enerji"].plot(kind="hist", bins=500)

In [ ]:
#  Calculate MAPE giving y_test and y_pred

def mean_absolute_percentage_error1(y_test, y_pred):
   
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test))*100

# Model XGBoost

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[(df_train1.tarih <= split_date)].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
xgb_model = XGBRegressor().fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# LightGBM

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[df_train1.tarih < split_date].copy()
df_train1_te = df_train1.loc[df_train1.tarih >= split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
lightgbm_model = LGBMRegressor().fit(X_train, y_train)
y_pred = lightgbm_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test,y_pred)

In [ ]:
# If you want to see variables importance level for LGBM
fig, ax = plt.subplots(figsize=(10,10))
Importance = pd.DataFrame({"Importance": lightgbm_model.feature_importances_*100},
                         index = X_train.columns)

Importance.sort_values(by = "Importance",
                       axis = 0,
                       ascending = True).plot(kind ="barh", color = "r", ax=ax)

plt.xlabel("Variable Importance Level ")
plt.show()

# CatBoost

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[df_train1.tarih <= split_date].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
catboost_model = CatBoostRegressor().fit(X_train, y_train)
y_pred = catboost_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

In [ ]:
# If you want to see variables importance level for CatBoost
fig, ax = plt.subplots(figsize=(10,10))
Importance = pd.DataFrame({"Importance": catboost_model.feature_importances_*100},
                         index = X_train.columns)

Importance.sort_values(by = "Importance",
                       axis = 0,
                       ascending = True).plot(kind ="barh", color = "r", ax=ax)

plt.xlabel("Variable Importance Level ")
plt.show()

# Linear Regression

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[df_train1.tarih <= split_date].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
lm_model = LinearRegression().fit(X_train, y_train)
y_pred = lm_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# GradientBoosting

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[df_train1.tarih <= split_date].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
gbm_model = GradientBoostingRegressor().fit(X_train, y_train)
y_pred = gbm_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# KNeighbors(KNN)

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[df_train1.tarih <= split_date].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)
y_pred = knn_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# Bagging

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[(df_train1.tarih <= split_date)].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
bagg_model = BaggingRegressor(bootstrap_features=True).fit(X_train, y_train)
y_pred = bagg_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# DecisionTree

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[(df_train1.tarih <= split_date)].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)


In [ ]:
tree_model = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = tree_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# MLP Regression

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[(df_train1.tarih <= split_date)].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
ml_model = MLPRegressor().fit(X_train, y_train)
y_pred = ml_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

# RandomForest

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[(df_train1.tarih <= split_date)].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
rf_model = RandomForestRegressor().fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

In [ ]:
# Comparison All the models which I create

models= [xgb_model,lightgbm_model,catboost_model,lm_model,gbm_model,knn_model,bagg_model,
        tree_model,ml_model,rf_model]
for model in models:
    names = model.__class__.__name__
    y_pred = model.predict(X_test)
    MAPE = mean_absolute_percentage_error1(y_test, y_pred)
    print("-" * 28)
    print(names + ":")
    print("MAPE: {:.4}".format(MAPE))

In [ ]:
result = []
results = pd.DataFrame(columns=["models", "MAPE"])

for model in models:
    names = model.__class__.__name__
    y_pred = model.predict(X_test)
    MAPE = mean_absolute_percentage_error1(y_test, y_pred)
    result = pd.DataFrame([[names, MAPE]], columns=["models", "MAPE"])
    results = results.append(result)
    results.sort_values(by="MAPE", ascending=False)

results.sort_values(by="MAPE", ascending=False)

sns.barplot(x='MAPE', y='models', data=results, color="r")
plt.xlabel('MAPE')
plt.title('Mean Absolute Percentage Error of Models');

### When I compare all the models that you can see above I decided to make model tuning for XGBoost

## Model Tuning XGBoost

In [ ]:
split_date = "2022-01-01"
df_train1_tr = df_train1.loc[(df_train1.tarih <= split_date)].copy()
df_train1_te = df_train1.loc[df_train1.tarih > split_date].copy()

y_train = df_train1_tr["enerji"]
X_train = df_train1_tr.drop(["enerji", "tarih"], axis=1)
y_test = df_train1_te["enerji"]
X_test = df_train1_te.drop(["enerji", "tarih"], axis=1)

In [ ]:
# Model tuning with best params
xgbm_tuned = XGBRegressor(colsample_bytree=0.6, learning_rate=0.1,
                          max_depth=6, n_estimators=2000, subsample=0.8).fit(X_train, y_train)

y_pred = xgbm_tuned.predict(X_test)

In [ ]:
mean_absolute_percentage_error1(y_test, y_pred)

In [ ]:
# Prediction

y1 = df_predict["enerji"]
X1 = df_predict.drop(['enerji', 'tarih'], axis=1)

df_predict["enerji"] = xgbm_tuned.predict(X1)
df_predict.head()
df_submission = pd.read_csv('/kaggle/input/gdz-elektrik-datathon-2023/sample_submission.csv')
df_submission["Dağıtılan Enerji (MWh)"] = df_predict["enerji"]
df_submission.head()
df_submission.index = df_submission["Tarih"]
df_submission.drop("Tarih", inplace=True, axis=1)
df_submission.to_csv("submission_sample_xgbm_tuned.csv")

In [ ]:
df_train1_te['predictions'] = y_pred

In [ ]:
# Analysing y_pred with y_test  
ax = df_train1[['enerji']].plot(figsize=(15, 5))
df_train1_te['predictions'].plot(ax=ax, style='.')
plt.legend(['Data', 'Predictions'])
ax.set_title('Data and Prediction')
plt.show()

In [ ]:
## If you want to see variables importance level for XGBoost_tuned
fig, ax = plt.subplots(figsize=(10,10))
Importance = pd.DataFrame({"Importance": xgbm_tuned.feature_importances_*100},
                         index = X_train.columns)

Importance.sort_values(by = "Importance",
                       axis = 0,
                       ascending = True).plot(kind ="barh", color = "r", ax=ax)

plt.xlabel("Variable Importance Level ")
plt.show()